# Is the Gallatin Canyon dry enough to  climb?

## Step 1: Mountain Project Ticks

In [567]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import dateutil
from timezonefinder import TimezoneFinder
import requests
from requests import RequestException
from bs4 import BeautifulSoup

In [ ]:
canyon_coords = [45.42, -111.23]

In [563]:
mp_base_url = "https://www.mountainproject.com"
# KEY = os.environ["MP_PRIVATE_KEY"]
KEY = "200335855-2c3a24cba7904fa7da08b088d31511d2"
TIMEZONE = TimezoneFinder().timezone_at(lat=canyon_coords[0], lng=canyon_coords[1])
tz = pytz.timezone(TIMEZONE)
TODAY = datetime.today().replace(tzinfo=tz)

In [ ]:
def get_routes_for_lat_long(lat, long, max_distance=None, max_results=None, min_diff=None, max_diff=None):
    params = {
        "lat": lat,
        "lon": long,
        "maxDistance": max_distance,
        "maxResults": max_results,
        "minDiff": min_diff,
        "maxDiff": max_diff,
        "key": KEY
    }
    params = {key: params[key] for key in params if params[key] is not None}
    res = requests.get(f"{mp_base_url}/data/get-routes-for-lat-lon", params=params)
    res.raise_for_status()
    return res.json()
    
    
def get_route(route_id):
    res = requests.get(f"{mp_base_url}/data/get-routes/", params={"routeIds": route_id, "key": KEY})
    res.raise_for_status()
    return res.json()
    
def get_routes(route_ids):
    res = requests.get(f"{mp_base_url}/data/get-routes/", params={"routeIds": route_ids, "key": KEY})
    res.raise_for_status()
    return res.json()

In [ ]:
gallatin_routes = get_routes_for_lat_long(*canyon_coords, max_distance=50, max_results=500)

In [ ]:
name_id_tuples = [(route["name"], route["id"]) for route in gallatin_routes["routes"]]

# Interlude: scrape for ticks
Mountain Project provides ticks per-user, not per-climb, from their API, so we have to go for the circuitous route.

In [ ]:
def get_html(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        resp = requests.get(url, stream=True)
        if is_good_response(resp):
            return resp.content
        else:
            return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
    
def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    print(e)

In [ ]:
parsed = []
for name, _id in name_id_tuples:
    try:
        url = f"{mp_base_url}/route/stats/{_id}"
        html = get_html(url)
        soup = BeautifulSoup(html, "html.parser")
        tables = soup.find_all("table")
        ticks = tables[3]
        data = ticks.find_all("tr")[0].find_all("td")
        date_string = " ".join(data[1].string.split(" ")[:3])
        date = datetime.strptime(date_string, "%b %d, %Y").replace(tzinfo=tz).isoformat()
        user_name = data[0].find("a").string
        user_id = data[0].find("a")["href"].split("/")[-2]
        parsed.append({"route_id": _id, "name": name, "date": date, "user_name": user_name, "user_id": user_id})
    except IndexError as e:
        print(f"Couldn't get that thing you wanted, skipping: {tables} \n Original error: {e}")

In [ ]:
ticks_table = pd.DataFrame(parsed)
# then save data

In [440]:
ticks_table.head()

,route_id,name,date,user_name,user_id
0,105961561,Sparerib,2020-05-29T00:00:00-07:00,Elle Olsztyn,200376334
1,105941845,Standard Route,2020-05-14T00:00:00-07:00,Sam Saarel,200539583
2,108077894,Flake Fest,2020-05-28T00:00:00-07:00,Blake Berghoff,200407744
3,106396564,Skyline Arete,2020-05-18T00:00:00-07:00,Kristen Neithercut,200537548
4,106870492,Tigger,2020-05-20T00:00:00-07:00,Aaron Day,200161465


In [ ]:
earliest_date = dateutil.parser.parse(ticks_table.date.min())

# Get Weather Data
https://api.mesowest.net/v2/stations/timeseries?stid=SGOM8&recent=18760&obtimezone=local&complete=1&hfmetars=0&token=d8c6aee36a994f90857925cea26934be

In [ ]:
weather_base = "https://api.mesowest.net/v2/stations"
TOKEN = "fec44906eebc49e5947a38b784e8907d"

In [ ]:
params = {
    "stid": "SGOM8",
    "recent": 18760,
    "obtimezone": "local",
    "complete": 1,
    "hfmetars": 0,
    "token": TOKEN
}
res = requests.get(f"{weather_base}/timeseries", params=params)

In [484]:
params = {
#     "stid": "SGOM8",
    "radius": ",".join([str(canyon_coords[0]), str(canyon_coords[1]), str(3)]),
    "start": earliest_date.strftime("%Y%m%d%H%M"),
    "end": datetime.today().strftime("%Y%m%d%H%M"),
    "token": TOKEN
}
res = requests.get(f"{weather_base}/timeseries", params=params)

In [523]:
params = {
    "radius": ",".join([str(canyon_coords[0]), str(canyon_coords[1]), str(50)]),
    "token": TOKEN
}
station_metadata = requests.get(f"{weather_base}/metadata", params=params)
df_station_metadata = pd.DataFrame(res.json()["STATION"])

In [544]:
df = df_station_metadata[df_station_metadata["STATUS"]=="ACTIVE"].drop(columns="STATUS")
df.columns = [col.lower() for col in df.columns]
df.head()
df = df[["id", "stid", "elevation", "distance", "longitude", "latitude", "state", "timezone"]]

In [657]:
def date_range(start, end, intv):
    diff = (end - start ) / intv
    for i in range(intv):
        yield (start + diff * i)
    yield end
        
def get_weather_data_chunks(start_datetime, end_datetime, lat, long):
    data = []
    for dt in date_range(start_datetime, end_datetime, 10):
        params = {
            "radius": ",".join([str(canyon_coords[0]), str(canyon_coords[1]), str(3)]),
            "start": start_datetime,
            "end": dt,
            "token": TOKEN
        }
        res = requests.get(f"{weather_base}/timeseries", params=params)
        res.raise_for_status()
        start_datetime = dt
        data.append(res.json())
        


In [658]:
# get_weather_data_chunks(earliest_date, datetime.today(), *canyon_coords)
get_weather_data_chunks(earliest_date, TODAY, *canyon_coords)


In [446]:
weather_df["station_id"] = "SGOM8"

In [447]:
weather_df.head()

,date_time,solar_radiation_set_1,wind_cardinal_direction_set_1d,wind_gust_set_1,volt_set_1,snow_interval_set_1,dew_point_temperature_set_1d,peak_wind_direction_set_1,wind_chill_set_1d,precip_accum_set_1,heat_index_set_1d,wind_direction_set_1,peak_wind_speed_set_1,wind_speed_set_1,relative_humidity_set_1,air_temp_set_1,station_id
0,2009-08-07T00:00:00Z,85.0,N,6.26,13.2,NaN,7.99,32.0,NaN,1058.164,NaN,354.0,6.26,2.68,43.0,21.11,SGOM8
1,2009-08-07T01:00:00Z,219.0,N,7.15,13.4,NaN,11.44,148.0,NaN,1058.164,NaN,9.0,7.15,2.23,74.0,16.11,SGOM8
2,2009-08-07T02:00:00Z,6.0,NE,3.58,13.0,NaN,11.56,102.0,NaN,1058.418,NaN,52.0,3.58,0.90,83.0,14.44,SGOM8
3,2009-08-07T03:00:00Z,4.0,SE,2.23,12.8,NaN,11.67,53.0,NaN,1061.466,NaN,141.0,2.23,0.45,100.0,11.67,SGOM8
4,2009-08-07T04:00:00Z,0.0,N,1.34,12.7,NaN,12.22,98.0,NaN,1061.466,NaN,9.0,1.34,0.00,100.0,12.22,SGOM8


# Save to S3